# 2019/09/22 ver.1.3

## 一、文件内容
特征选择：GBRT+RT+XGboost取平均值 选择前380个特征<br />
单模型构建：<br />
 1：SVR<br />
 2：RF<br />
 3：XGBoost<br />
 4：KNN<br />
 5：GBRT<br />
Stacking：<br />
 XGboost默认
## 二、数据集
### 导入数据集变量<br />
元数据集 训练集全部：dg_train →训练集 特征数据：dg_train<br />
元数据集 测试集全部：dg_test  →测试集 特征数据：dg_test<br />
<br />
训练集 得分数据：score_train<br />
测试集 序号：id_test<br />
训练集 序号：id_train
<br />
### 处理数据集变量<br />
标准化处理后的原训练集 特征数据：dg_scaled_train<br />
标准化处理后的测试集合 特征数据：dg_scaled_test<br />
### 特征选择结果
特征选择后的训练集 特征数据：X<br />
特征选择后的测试集 特征数据：X_predict<br />
### 在X中继续划分数据<br />
X_train：0.8比例的原训练集特征 用作训练和验证<br />
X_test：0.2比例的原训练集特征 用作测试（模型选择）<br />
y_train：0.8比例的原训练集分数 对应X_train<br />
y_test：0.2比例的原训练集分数 对应X_test<br />

## 三、模型信息


变量名|模型名|超参数设置|随机数种子|训练集上预测值
:---------:|:------------------:|:-----------------:|:-----------------:|:-------------------------:
model_svc|支持向量回归||NA|
model_svcRF|随机森林回归||160|
model_xgbr|Xgboost回归||161|
model_k_neighbor|KNN回归||NA|
model_gradient_boosting_regressor|GBRT||162|

## 四、随机数信息
位置|使用函数|随机数种子
:------:|:-------:|:--------:|
特征处理|GBRT|20
特征处理|train_test_split|21
stacking|KFold|22
预测|train_test_split|23


# 1、前処理（preprocess）

In [1]:
#预设导入
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import joblib


import warnings
warnings.filterwarnings("ignore")

In [2]:
#机器学习导入
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import r2_score
from sklearn import ensemble
from sklearn import neighbors
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
dg_train=pd.read_csv('train_all.csv')
dg_test=pd.read_csv('test_all.csv')

#单独提取ID列和score列
id_train=dg_train['id'].values
id_test=dg_test['id'].values
score_train=dg_train['SalePrice'].values

dg_train=dg_train.set_index(['id'])
dg_test=dg_test.set_index(['id'])


del dg_train['SalePrice']


In [41]:
dg_test

,1,2,3,4,5,6,7,8,9,10,...,446,447,448,449,450,451,452,453,454,455
id,,,,,,,,,,,,,,,,,,,,,
31471,0.439706,-1.563669,2.886859,2.457511,0.599518,-1.694717,-0.843369,-1.067786,-2.975376,-3.046322,...,-0.03826,-1.119594,-0.289206,-0.065881,-0.009764,-0.460807,-0.247177,2.728746,-0.093518,0.819145
31472,0.511786,-1.901381,2.948915,0.880258,1.253064,-2.859232,-1.944730,-0.360820,-1.962810,-4.539415,...,-0.03826,0.893181,-0.289206,-0.065881,-0.009764,-0.460807,-0.247177,-0.366469,-0.093518,-0.740336
31473,0.452382,-2.082961,-3.959528,0.669883,-0.959463,0.704863,-3.940546,-0.474106,-1.147783,0.927036,...,-0.03826,-1.119594,-0.289206,-0.065881,-0.009764,-0.460807,-0.247177,2.728746,-0.093518,-0.418766
31474,-0.454103,0.311770,1.112790,2.839259,-1.190529,1.093948,0.297414,-0.480660,2.969900,0.890799,...,-0.03826,-1.119594,-0.289206,-0.065881,-0.009764,2.170106,-0.247177,-0.366469,-0.093518,-0.597357
31475,-1.371399,1.319441,0.510782,0.364321,-1.876332,0.834969,-0.884610,-0.230953,2.975468,-1.446590,...,-0.03826,0.893181,-0.289206,-0.065881,-0.009764,-0.460807,-0.247177,-0.366469,-0.093518,0.807452
31476,-2.037444,1.538298,-0.562356,0.434211,-1.824582,-0.632189,0.647655,0.537794,3.397825,-1.452640,...,-0.03826,-1.119594,-0.289206,-0.065881,-0.009764,2.170106,-0.247177,-0.366469,-0.093518,-0.710040
31477,0.016683,-1.561418,-2.045769,-0.856644,1.407410,-0.880144,-0.391444,-1.950523,0.961449,-1.081100,...,-0.03826,0.893181,-0.289206,-0.065881,-0.009764,-0.460807,-0.247177,-0.366469,-0.093518,-0.788173
31478,-0.844482,0.609301,0.248616,2.494277,-0.872695,0.352762,1.181683,0.325032,3.930930,0.354652,...,-0.03826,-1.119594,-0.289206,-0.065881,-0.009764,2.170106,-0.247177,-0.366469,-0.093518,-0.551115
31479,-2.685328,3.678278,1.920848,-2.142959,-1.302532,2.120069,-2.131156,-0.407303,-1.033841,-0.307280,...,-0.03826,-1.119594,-0.289206,-0.065881,-0.009764,2.170106,-0.247177,-0.366469,-0.093518,0.080331


In [43]:
dg_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31262 entries, 31471 to 62732
Columns: 455 entries, 1 to 455
dtypes: float64(455)
memory usage: 108.8 MB


In [8]:
pd.set_option('max_colwidth',200)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows',50)

In [4]:
temp = dg_test.isnull().any() #列中是否存在空值
print(type(temp))
print(temp)

<class 'pandas.core.series.Series'>
1      False
2      False
3      False
4      False
5      False
       ...  
451    False
452    False
453    False
454    False
455    False
Length: 455, dtype: bool


In [10]:
#z-score 标准化
scaler=StandardScaler().fit(dg_train)#标准化的mean var
print(scaler)
print(scaler.mean_)
print(scaler.var_)
dg_scaled_train=scaler.transform(dg_train)#标准化结果向量
print(dg_scaled_train)
np.shape(dg_scaled_train)

dg_scaled_test=scaler.transform(dg_test)
print(dg_scaled_test)
np.shape(dg_scaled_test)#相同标准 标准化测试集

StandardScaler(copy=True, with_mean=True, with_std=True)
[-9.84418916e-17  1.12892077e-17  9.48293451e-17 -1.02054438e-16
 -6.50258366e-17  1.34567356e-16  4.69631042e-17  2.21268472e-17
  7.36056345e-17  3.97380113e-17 -1.26439127e-17  1.06570121e-16
 -1.11537373e-16  6.32195634e-18 -1.62564592e-17 -7.76697493e-17
 -4.28989894e-17 -2.93519401e-17 -3.56738965e-17 -7.67666127e-18
  4.83178092e-17 -1.14246782e-16  1.49017542e-16  1.29374321e-16
  6.84125990e-17  3.52223282e-17  6.23164268e-17 -8.12822958e-18
 -2.25784155e-17 -6.66063257e-17 -1.71595958e-17 -1.11085804e-16
  1.64822433e-17 -2.93519401e-17  3.92864430e-17  2.61909620e-17
  3.83833063e-17  6.50258366e-17 -2.70940986e-17 -4.96725141e-17
 -1.26439127e-17  4.17700687e-17  0.00000000e+00  2.00947898e-17
 -8.08307275e-17 -7.47345553e-17  3.70286014e-17  6.77352465e-18
  5.46397655e-17  6.52516208e-17  8.26370007e-17  2.77714511e-17
  3.52223282e-17  3.38676232e-17 -3.58996806e-17  1.53533225e-17
  6.90899514e-17  1.08376394e-17 

  -0.72439072]]


(31262, 455)

In [5]:
dg_scaled_train=dg_train.values
dg_scaled_test=dg_test.values

In [6]:
dg_scaled_train

array([[ 1.39403375, -2.42505996, -4.49437984, ..., -0.3664687 ,
        -0.09351828, -0.78657864],
       [-1.00121151,  0.19291356, -2.23135782, ...,  2.728746  ,
        -0.09351828, -0.97314237],
       [-0.12704885, -0.01553204,  1.47476304, ..., -0.3664687 ,
        -0.09351828, -0.67814843],
       ...,
       [-1.33092903,  1.20629013,  0.0638261 , ...,  2.728746  ,
        -0.09351828,  1.33897231],
       [-2.25548404,  2.28327712,  0.77140135, ..., -0.3664687 ,
        -0.09351828, -0.27738188],
       [-0.77335044, -0.25956873, -2.67405341, ...,  2.728746  ,
        -0.09351828, -0.52134985]])

In [ ]:
sns.distplot(score_train)

### 以下开始特征选择

In [ ]:
feature_importance = clf_fs.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())


fi_threshold = 18    
important_idx = np.where(feature_importance > fi_threshold)[0]
important_features = features_list[important_idx]


    print("\n", important_features.shape[0], "Important features(>", \
          fi_threshold, "% of max importance)...\n")
            #important_features
    sorted_idx = np.argsort(feature_importance[important_idx])[::-1]
    #get the figure about important features
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.subplot(1, 2, 2)
    plt.title('Feature Importance')
    plt.barh(pos, feature_importance[important_idx][sorted_idx[::-1]], \
            color='r',align='center')
    plt.yticks(pos, important_features[sorted_idx[::-1]])
    plt.xlabel('Relative Importance')
    plt.draw()
plt.show()

In [7]:
clf_fs_1 = ensemble.GradientBoostingRegressor(random_state=120)
clf_fs_1 =clf_fs_1.fit(dg_scaled_train, score_train)
clf_fs_1.feature_importances_  


array([2.65308893e-03, 6.87972795e-04, 7.98705089e-05, 2.27842639e-04,
       1.46403215e-02, 1.17930950e-03, 4.19014453e-04, 1.18219355e-02,
       2.03191060e-03, 2.37593988e-04, 2.09931462e-03, 3.76941547e-04,
       0.00000000e+00, 1.19562443e-04, 6.24221393e-05, 2.69610041e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.40881395e-02, 0.00000000e+00, 3.17013300e-03, 1.97824946e-05,
       4.99777447e-04, 0.00000000e+00, 6.09020318e-05, 3.99213480e-04,
       2.99983727e-04, 2.96118870e-04, 9.43223185e-04, 0.00000000e+00,
       1.19067177e-03, 0.00000000e+00, 4.15141401e-03, 0.00000000e+00,
       3.14848541e-04, 0.00000000e+00, 0.00000000e+00, 1.20072690e-03,
       5.05467446e-05, 1.01156895e-03, 0.00000000e+00, 8.10979885e-04,
       0.00000000e+00, 0.00000000e+00, 1.86291664e-04, 6.89438166e-05,
       1.87533182e-04, 0.00000000e+00, 3.02425856e-04, 1.09734144e-04,
       1.82440130e-04, 9.39934112e-04, 0.00000000e+00, 0.00000000e+00,
      

In [15]:
clf_fs_2 = RandomForestRegressor(random_state=150)
clf_fs_2 =clf_fs_2.fit(dg_scaled_train, score_train)
clf_fs_2.feature_importances_ 

array([5.56973137e-03, 5.23677149e-03, 1.95089858e-03, 1.49786924e-03,
       8.61559335e-03, 2.09932864e-03, 1.05629663e-03, 3.95588398e-03,
       3.61580852e-03, 6.36115711e-04, 3.89773453e-04, 1.21090859e-03,
       4.19374946e-04, 5.26169368e-04, 9.09901180e-04, 7.63527595e-04,
       9.81813608e-04, 2.36605302e-04, 5.73178138e-04, 8.07095642e-04,
       4.24256597e-03, 4.84334951e-03, 5.94364135e-03, 3.62061574e-04,
       8.89683729e-04, 1.41843082e-03, 3.47318002e-04, 1.31193146e-03,
       5.68636319e-04, 5.91936549e-04, 2.96343579e-04, 3.76104785e-04,
       3.28966964e-04, 3.02123450e-04, 7.41348409e-03, 2.76394542e-04,
       2.97538574e-03, 5.09704365e-04, 1.40631157e-03, 2.06312184e-03,
       6.84847074e-04, 5.99204103e-04, 5.59722408e-04, 2.64486154e-04,
       4.81862824e-04, 5.90611236e-04, 1.88978844e-04, 6.91439949e-04,
       2.91190863e-04, 1.28102856e-03, 4.28018909e-04, 2.96990561e-04,
       4.93717525e-04, 5.99766507e-04, 6.17614530e-04, 8.72921638e-04,
      

In [8]:
clf_fs_3 = xgb.XGBRegressor(random_state=151)
clf_fs_3 =clf_fs_3.fit(dg_scaled_train, score_train)
clf_fs_3.feature_importances_  

[16:14:39] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([0.0039762 , 0.0056835 , 0.01634524, 0.00157798, 0.02455863,
       0.00721036, 0.00232874, 0.01604441, 0.00870163, 0.00530214,
       0.02031631, 0.00389328, 0.        , 0.00269644, 0.0025021 ,
       0.        , 0.        , 0.0034817 , 0.00322748, 0.00289669,
       0.02975027, 0.00078814, 0.02763163, 0.        , 0.0038662 ,
       0.        , 0.        , 0.00453342, 0.00311516, 0.0016042 ,
       0.01546049, 0.        , 0.00184897, 0.        , 0.00815811,
       0.        , 0.00178288, 0.00723859, 0.        , 0.02439605,
       0.        , 0.00619278, 0.        , 0.00285307, 0.        ,
       0.        , 0.        , 0.        , 0.00158517, 0.        ,
       0.0025359 , 0.00136969, 0.00140693, 0.00381462, 0.        ,
       0.00091361, 0.        , 0.        , 0.00190803, 0.00487874,
       0.        , 0.        , 0.02906667, 0.        , 0.        ,
       0.        , 0.        , 0.00069306, 0.        , 0.        ,
       0.00725177, 0.00303634, 0.00244387, 0.02058104, 0.00243

In [9]:
feature_importance_1=clf_fs_1.feature_importances_
#feature_importance_2=clf_fs_2.feature_importances_
feature_importance_3=clf_fs_3.feature_importances_
feature_importance=np.add(feature_importance_1,feature_importance_3)


In [12]:
np.sort(feature_importance)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [10]:
#feature_importance 170维
index_fs=np.argsort(-feature_importance)
index_fs_select=index_fs[0:170]
X=dg_scaled_train[:,index_fs_select]
X_predict=dg_scaled_test[:,index_fs_select]

In [14]:
print(index_fs_select)

[454 430 422 441 431  20 117 453   4 446 421  62  22   7  73  39  10 428
   2  30 124  34 363 407   8 120 410 395 440  70 257 333 320   5 281 391
 419  37  41   0 315   1 429   9 255  59 119  27 423 107  53  24 397  11
 426 358 386 201 111  43 376  17  78  28 264  71 130  18 220  32  19 219
  50  13 101  72 156   6 313 261  74  14 362 176 244 157 290  36 213 304
  94  58 150  29 300   3 424  48 418 254  77  52 167 324 137 323 416 198
 121  51 299 214 204 442 138 272 450  81 230  55 325  69 380 249 311 307
  21  67  76 452 164 399 223 154 131  15 153 309 357 366  60 232 387 344
  46 334 172  97 145 160 236 393 173 237 231  47 337 351  26 316 369  40
 189 139 100  23 435 104 238 390]


In [14]:
np.shape(X)

(31470, 170)

In [ ]:
pd.DataFrame(dg_scaled_train).describe()

In [ ]:
#不运行
#基模型的特征选择
clf_fs = ensemble.GradientBoostingRegressor(random_state=20)
clf_fs =clf_fs.fit(dg_scaled_train, score_train)
clf_fs.feature_importances_  

model_fs = SelectFromModel(clf_fs, prefit=True)
X = model_fs.transform(dg_scaled_train)


#SelectFromModel(ensemble.GradientBoostingRegressor()).fit_transform(dg_scaled_train, score_train)

X_predict=model_fs.transform(dg_scaled_test)
print(dg_scaled_test)
np.shape(X_predict)

In [11]:
#划分数据集

X_train, X_test, y_train, y_test = train_test_split(X, score_train, test_size=0.2, random_state=21)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((25176, 170), (25176,), (6294, 170), (6294,))

In [ ]:


rms = sqrt(mean_squared_error(y_actual, y_predicted))


# 2、学習（train）

In [12]:
#
from sklearn import linear_model
model_reg = linear_model.Ridge()
model_reg.fit(X_train,y_train)

print(model_reg.score(X_test,y_test))

y_pred = model_reg.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))

print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

0.7316969922445721
0.7316969922445721
38590.827874245486


In [13]:
from sklearn import linear_model
model_reg = linear_model.LinearRegression()
model_reg.fit(X_train,y_train)

print(model_reg.score(X_test,y_test))

y_pred = model_reg.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))

print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

0.7316845191843815
0.7316845191843815
38591.72488268522


In [15]:
from sklearn import linear_model
model_reg = linear_model.LassoLars()
model_reg.fit(X_train,y_train)

print(model_reg.score(X_test,y_test))

y_pred = model_reg.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))

print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

0.7328717036698977
0.7328717036698977
38506.25406309121


In [16]:
from sklearn import linear_model
model_reg = linear_model.BayesianRidge()
model_reg.fit(X_train,y_train)

print(model_reg.score(X_test,y_test))

y_pred = model_reg.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))

print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)


0.7321818676008225
0.7321818676008225
38555.94156575715


### SVR

In [46]:
#SVR模型实现
model_svc=svm.SVR()
model_svc.fit(X_train,y_train)

print(model_svc.score(X_test,y_test))

y_pred = model_svc.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))

print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

-0.04144854451134572
-0.04144854451134572
76030.9178712978


GBDT 0.5903171534507402<br>
RF 0.5714306438983159<br >
XBG 0.5864080375808052<br>
平均500 0.5877608712908593<br>
平均400 0.5889615497881555<br>
358 0.5888839063663873<br>
370 0.5889239406579279<br>
375 0.589024306528543<br>
380 0.5897356830942392<br>


In [ ]:
#SVR模型调参数：C gamma
param_g=[{"C": [1e0, 1e1, 1e2, 1e3],"gamma": np.logspace(-4, 0, 5)}]
scores_g=['r2']
for score in scores_g:
    print(score)
    clf=GridSearchCV(svm.SVR(),param_grid=param_g,cv=3,scoring=score)
    clf.fit(X_train,y_train)
    
    print(clf.best_estimator_)
    
    means = clf.cv_results_['mean_test_score']
    stds=clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

In [ ]:
#SVR模型实现
model_svc_t=clf.best_estimator_
model_svc_t.fit(X_train,y_train)

print(model_svc_t.score(X_test,y_test))

y_pred = model_svc_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

model_svc=clf.best_estimator_

### RF

In [19]:
model_svcRF=RandomForestRegressor(n_estimators=200,random_state=160)
model_svcRF.fit(X_train,y_train)

print(model_svcRF.score(X_test,y_test))#0.4697676007383277

y_pred = model_svcRF.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

0.8681192598059411
0.8681192598059411
27055.90172620822


380 0.4808578016857748

In [38]:
N_estimators=[{"n_estimators":[150,200,300,400,500]}]#20,50,100,
scores_g=['r2']
for score in scores_g:
    print(score)
    clf=GridSearchCV(RandomForestRegressor(random_state=160),param_grid=N_estimators,cv=3,scoring=score)
    clf.fit(X_train,y_train)
    
    print(clf.best_estimator_)
    
    means = clf.cv_results_['mean_test_score']
    stds=clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

r2


KeyboardInterrupt: 

r2
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=160, verbose=0,
                      warm_start=False)
0.881(+/-0.027) for {'n_estimators': 20}
0.888(+/-0.029) for {'n_estimators': 50}
0.888(+/-0.029) for {'n_estimators': 100}
0.889(+/-0.030) for {'n_estimators': 150}
0.890(+/-0.029) for {'n_estimators': 200}

In [22]:
model_svcRF_t=clf.best_estimator_
model_svcRF_t.fit(X_train,y_train)

print(model_svcRF_t.score(X_test,y_test))#默认0.4697676007383277

y_pred = model_svcRF_t.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)
model_svcRF=clf.best_estimator_

0.8765851011968508
0.8765851011968508
26173.097349764343


### xgb

In [31]:
model_xgbr = xgb.XGBRegressor(n_estimators=500,max_depth=5,min_child_weight= 3,learning_rate=0.1,random_state=161)
model_xgbr.fit(X_train, y_train)

print(model_xgbr.score(X_test,y_test))

y_pred = model_xgbr.predict(X_test)

rms = sqrt(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

[13:27:09] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9132798872132086
0.9132798872132086
21939.728138616727


[15:59:54] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.8787590706564508
0.8787590706564508
25941.551890190498

21803

In [26]:
#调参 XGboost——n_estimators 其他默认
cv_params = [{"n_estimators":[20,50,100,150,200,300,400,500]}]

scores_GBM=['r2']

for score in scores_GBM:
    print(score)
    model = XGBRegressor(random_state=161)
    optimized_GBM=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM.fit(X_train,y_train)
    
    print(optimized_GBM.best_estimator_)
    
    means_GBM = optimized_GBM.cv_results_['mean_test_score']
    stds_GBM=optimized_GBM.cv_results_['std_test_score']
    params_GBM = optimized_GBM.cv_results_['params']
    
    for (param_GBM,mean_score_GBM,std_score_GBM) in zip(params_GBM,means_GBM,stds_GBM):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM,std_score_GBM,param_GBM))

r2
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  3.9min finished


[12:40:31] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
0.716(+/-0.024) for {'n_estimators': 20}
0.859(+/-0.023) for {'n_estimators': 50}
0.889(+/-0.022) for {'n_estimators': 100}
0.898(+/-0.020) for {'n_estimators': 150}
0.903(+/-0.020) for {'n_estimators': 200}
0.908(+/-0.018) for {'n_estimators': 300}
0.911(+/-0.018) for {'n_estimators': 400}
0.913(+/-0.017) for {'n_estimators': 500}


In [29]:
#调参 XGboost——maxdepth minchildweight 其他默认
cv_params = [{'max_depth': [3, 5,7,9], 'min_child_weight': [1, 3, 5]}]

scores_GBM_mami=['r2']

for score in scores_GBM_mami:
    print(score)
    model = optimized_GBM.best_estimator_
    optimized_GBM_mami=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM_mami.fit(X_train,y_train)
    
    print(optimized_GBM_mami.best_estimator_)
    
    means_GBM_mami = optimized_GBM_mami.cv_results_['mean_test_score']
    stds_GBM_mami=optimized_GBM_mami.cv_results_['std_test_score']
    params_GBM_mami = optimized_GBM_mami.cv_results_['params']
    
    for (param_GBM_mami,mean_score_GBM_mami,std_score_GBM_mami) in zip(params_GBM_mami,means_GBM_mami,stds_GBM_mami):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM_mami,std_score_GBM_mami,param_GBM_mami))

r2
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 29.2min finished


[13:16:59] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
0.913(+/-0.017) for {'max_depth': 3, 'min_child_weight': 1}
0.913(+/-0.018) for {'max_depth': 3, 'min_child_weight': 3}
0.914(+/-0.022) for {'max_depth': 3, 'min_child_weight': 5}
0.917(+/-0.017) for {'max_depth': 5, 'min_child_weight': 1}
0.918(+/-0.017) for {'max_depth': 5, 'min_child_weight': 3}
0.917(+/-0.020) for {'max_depth': 5, 'min_child_weight': 5}
0.913(

In [32]:
#调参 XGboost——学习率
cv_params = [{'learning_rate':  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]}]

scores_GBM_lr=['r2']

for score in scores_GBM_lr:
    print(score)
    model = optimized_GBM_mami.best_estimator_
    optimized_GBM_lr=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM_lr.fit(X_train,y_train)
    
    print(optimized_GBM_lr.best_estimator_)
    
    means_GBM_lr = optimized_GBM_lr.cv_results_['mean_test_score']
    stds_GBM_lr=optimized_GBM_lr.cv_results_['std_test_score']
    params_GBM_lr = optimized_GBM_lr.cv_results_['params']
    
    for (param_GBM_lr,mean_score_GBM_lr,std_score_GBM_lr) in zip(params_GBM_lr,means_GBM_lr,stds_GBM_lr):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM_lr,std_score_GBM_lr,param_GBM_lr))

r2
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 12.7min finished


[13:42:08] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
-2.290(+/-0.136) for {'learning_rate': 0.0001}
-0.493(+/-0.042) for {'learning_rate': 0.001}
0.898(+/-0.022) for {'learning_rate': 0.01}
0.918(+/-0.017) for {'learning_rate': 0.1}
0.916(+/-0.015) for {'learning_rate': 0.2}
0.908(+/-0.023) for {'learning_rate': 0.3}


In [ ]:
model_xgbr_t = optimized_GBM.best_estimator_
model_xgbr_t.fit(X_train, y_train)

print(model_xgbr_t.score(X_test,y_test))

y_pred = model_xgbr_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
model_xgbr = optimized_GBM.best_estimator_

### KNN

In [37]:
# 4.kNN回归
model_k_neighbor = neighbors.KNeighborsRegressor(n_neighbors=4, p=5,weights='distance')
model_k_neighbor.fit(X_train, y_train)

print(model_k_neighbor.score(X_test,y_test))

y_pred = model_k_neighbor.predict(X_test)
rms = sqrt(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

0.7112508524026153
0.7112508524026153
40034.248464909884


In [35]:
param_grids=[
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1,11)]
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1,11)],
        'p':[i for i in range(1,6)]
    }]
scores_KNN=['r2']

for score in scores_KNN:
    print(score)
    model = neighbors.KNeighborsRegressor()
    clf_k=GridSearchCV(estimator=model, param_grid=param_grids, scoring=score, cv=3, verbose=1, n_jobs=-1)
    clf_k.fit(X_train,y_train)
    
    print(clf_k.best_estimator_)
    
    means_KNN = clf_k.cv_results_['mean_test_score']
    stds_KNN=clf_k.cv_results_['std_test_score']
    params_KNN = clf_k.cv_results_['params']
    
    for (param_KNN,mean_score_KNN,std_score_KNN) in zip(params_KNN,means_KNN,stds_KNN):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_KNN,std_score_KNN,param_KNN))

r2
Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 83.9min finished


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=4, p=5,
                    weights='distance')
0.558(+/-0.026) for {'n_neighbors': 1, 'weights': 'uniform'}
0.586(+/-0.004) for {'n_neighbors': 2, 'weights': 'uniform'}
0.584(+/-0.014) for {'n_neighbors': 3, 'weights': 'uniform'}
0.573(+/-0.022) for {'n_neighbors': 4, 'weights': 'uniform'}
0.558(+/-0.023) for {'n_neighbors': 5, 'weights': 'uniform'}
0.547(+/-0.022) for {'n_neighbors': 6, 'weights': 'uniform'}
0.535(+/-0.025) for {'n_neighbors': 7, 'weights': 'uniform'}
0.525(+/-0.025) for {'n_neighbors': 8, 'weights': 'uniform'}
0.515(+/-0.027) for {'n_neighbors': 9, 'weights': 'uniform'}
0.507(+/-0.026) for {'n_neighbors': 10, 'weights': 'uniform'}
0.337(+/-0.035) for {'n_neighbors': 1, 'p': 1, 'weights': 'distance'}
0.558(+/-0.026) for {'n_neighbors': 1, 'p': 2, 'weights': 'distance'}
0.612(+/-0.012) for {'n_neighbors': 1, 'p': 3, 'weights': 'dista

In [ ]:
model_k_neighbor_t = clf_k.best_estimator_
model_k_neighbor_t.fit(X_train, y_train)

print(model_k_neighbor_t.score(X_test,y_test))

y_pred = model_k_neighbor_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

model_k_neighbor = clf_k.best_estimator_

### GBRT

In [34]:
model_gradient_boosting_regressor = ensemble.GradientBoostingRegressor(n_estimators=500,random_state=162)  

model_gradient_boosting_regressor.fit(X_train, y_train)

print(model_gradient_boosting_regressor.score(X_test,y_test))

y_pred = model_gradient_boosting_regressor.predict(X_test)
rms = sqrt(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

0.9014748261569127
0.9014748261569127
23385.40683455488


In [33]:
N_estimators=[{"n_estimators":[20,50,100,150,200,300,400,500]}]
scores_g=['r2']
for score in scores_g:
    print(score)
    clf_gb=GridSearchCV(ensemble.GradientBoostingRegressor(random_state=162),param_grid=N_estimators,cv=3,scoring=score)
    clf_gb.fit(X_train,y_train)
    
    print(clf_gb.best_estimator_)
    
    means = clf_gb.cv_results_['mean_test_score']
    stds=clf_gb.cv_results_['std_test_score']
    params = clf_gb.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

r2
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='auto',
                          random_state=162, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
0.758(+/-0.025) for {'n_estimators': 20}
0.859(+/-0.025) for {'n_estimators': 50}
0.888(+/-0.024) for {'n_estimators': 100}
0.897(+/-0.023) for {'n_estimators': 150}
0.901(+/-0.021) for {'n_estimators': 200}
0.904(+/-0.019) for {'n_estimators': 300}
0.907(+/-0.018) for {'n_estimators': 400}
0.909(+/-0.019) for {'n_estimators': 500}


In [ ]:
model_gradient_boosting_regressor_t = clf_gb.best_estimator_

model_gradient_boosting_regressor_t.fit(X_train, y_train)

print(model_gradient_boosting_regressor_t.score(X_test,y_test))

y_pred = model_gradient_boosting_regressor_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

model_gradient_boosting_regressor = clf_gb.best_estimator_

# 第二阶段stacking

In [47]:
# '''创建训练的数据集'''
#data, target = make_blobs(n_samples=50000, centers=2, random_state=0, cluster_std=0.60)
 
# '''模型融合中使用到的各个单模型'''
# clfs = [RandomForestRegressor(n_estimators=200,random_state=160),
#         xgb.XGBRegressor(n_estimators=500,max_depth=5,min_child_weight= 3,learning_rate=0.1,random_state=161),
#         neighbors.KNeighborsRegressor(n_neighbors=4, p=5,weights='distance'),
#         ensemble.GradientBoostingRegressor(n_estimators=500,random_state=162)
#        ]

clfs=[linear_model.LassoLars()]
 
#'''切分一部分数据作为测试集'''
X=X#训练集 数据
X_predict=X_predict#测试集 数据
y=score_train#训练集 分数
#y_predict = 


dataset_blend_train = np.zeros((X.shape[0], len(clfs)))#第一轮 保存各个模型在训练集上的预测结果 训练集合个数×模型数
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))#第一轮 保存各个模型在测试集上的预测结果 训练集合个数×模型数

#'''5折stacking'''
n_folds = 5
kf = KFold(n_folds,True,22)
skf=list(kf.split(X))#X或者y

for j, clf in enumerate(clfs):
    #'''依次训练各个单模型'''
    print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))#存目前这个模型上的测试集结果(之后求平均)
    for i, (train, test) in enumerate(skf):
        #'''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict(X_test)#1fold的预测结果
        y_submission=y_submission.flatten()#2维数组转1维 KNN需要 
        #y_submission.reshape(len(y_submission),1)#一维数组转二维 可以不加
        
        dataset_blend_train[test, j] = y_submission#在模型顺序对应的j位置 存1fold的预测结果
        dataset_blend_test_j[:, i] = clf.predict(X_predict).flatten()#存该模型该折下的测试集预测结果
        
    #'''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)#测试集结果按行取平均后储存
    
    
    #print("val auc Score: %f" % r2_score(y_predict, dataset_blend_test[:, j]))
    

0 LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, max_iter=500, normalize=True, positive=False,
          precompute='auto', verbose=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [18]:
#导出数据集到本地
submission_train_1=pd.DataFrame(dataset_blend_train)
#submission_train_1.head()
submission_train_1.to_csv('dataset_blend_train.csv',index=False)#第一轮训练后 train集合预测得到的score集合 训练集样本数x3个模型

submission_test_1=pd.DataFrame(dataset_blend_test)
#submission_test_1.head()
submission_test_1.to_csv('dataset_blend_test.csv',index=False)#第一轮训练后 test集合预测得到的score集合 测试机样本数x3个模型

In [28]:
dataset_blend_train

array([[ 69767.87412296,  67766.24768285],
       [ 73157.26412891,  72440.8071711 ],
       [138781.99988668, 138958.9755904 ],
       ...,
       [143100.38845517, 142856.80655515],
       [ 97637.71248373,  98126.75807216],
       [ 66485.08161537,  68580.36251023]])

In [40]:
submission_train_1

,0,1,2,3,0
0,89937.500000,84493.390625,108429.109267,79433.521061,69767.874123
1,62443.650000,69395.703125,64634.334669,74665.063106,73157.264129
2,106437.500000,106312.164062,120705.589000,108668.400088,138781.999887
3,161030.000000,165868.609375,132800.298222,167444.237734,174284.761975
4,69494.600000,68470.320312,50971.584571,64731.329705,69556.103052
5,177290.500000,158107.656250,115982.129181,159979.370161,138019.289778
6,83135.000000,80776.789062,63887.549592,78928.931018,70819.427851
7,121170.000000,126021.359375,115000.000000,130665.155413,125717.205029
8,159016.000000,171063.640625,89336.987986,164552.544890,180415.592093
9,73365.000000,75764.335938,74780.450806,75330.402797,73525.936277


In [46]:
submission_test_1

,0,1,2,3,0
0,155050.600000,145980.575000,105068.387634,149208.125516,69767.874123
1,114510.900000,115455.056250,121635.602127,116022.071633,73157.264129
2,81643.404333,77090.054688,84493.311487,75629.253182,138781.999887
3,59682.300000,60101.988281,61453.743495,60992.329779,174284.761975
4,120968.511905,127010.950000,95323.079347,132652.352853,69556.103052
5,66154.466667,72803.907812,100329.761211,69111.170237,138019.289778
6,94263.400000,89360.315625,117596.418753,99250.824806,70819.427851
7,89537.800000,91857.209375,73326.588462,91624.620988,125717.205029
8,94743.150000,94670.592188,66202.089772,91200.873716,180415.592093
9,133978.400000,122991.323438,109126.435271,126428.406598,73525.936277


In [48]:
submission_train_1=pd.read_csv('dataset_blend_train.csv')
submission_test_1=pd.read_csv('dataset_blend_test.csv')

In [49]:
a=pd.DataFrame(dataset_blend_train)
submission_train_1=pd.concat([submission_train_1,a],axis=1)

In [50]:
b=pd.DataFrame(dataset_blend_test)
submission_test_1=pd.concat([submission_test_1,b],axis=1)

In [51]:
dataset_blend_train=submission_train_1.values
dataset_blend_test=submission_test_1.values

# 3、予測（predict）

In [56]:
#第二轮 
model_stacking_xgb = optimized_GBM.best_estimator_##xgb.XGBRegressor(random_state=200)
model_stacking_xgb.fit(dataset_blend_train, y)


#model_stacking_svm=svm.SVR()
#model_stacking_svm.fit(dataset_blend_train, y)

y_submission = model_stacking_xgb.predict(dataset_blend_test)

#生成文件
submission_df=pd.DataFrame(data={'id':id_test,'SalePrice':y_submission})
submission_df.head()
submission_df.to_csv('baseline_2_v.csv',header=False,index=False)

[16:42:40] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
rms = sqrt(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

In [54]:
#调参数
X_train, X_test, y_train, y_test = train_test_split(dataset_blend_train, y, test_size=0.2, random_state=123)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

#调参 XGboost——n_estimators 其他默认
cv_params = [{"n_estimators":[20,50,100,150,200,300,400,500]}]

scores_GBM=['r2']

for score in scores_GBM:
    print(score)
    model = XGBRegressor(random_state=161)
    optimized_GBM=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM.fit(X_train,y_train)
    
    print(optimized_GBM.best_estimator_)
    
    means_GBM = optimized_GBM.cv_results_['mean_test_score']
    stds_GBM=optimized_GBM.cv_results_['std_test_score']
    params_GBM = optimized_GBM.cv_results_['params']
    
    for (param_GBM,mean_score_GBM,std_score_GBM) in zip(params_GBM,means_GBM,stds_GBM):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM,std_score_GBM,param_GBM))

(25176, 5) (25176,) (6294, 5) (6294,)
r2
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   14.8s finished


[16:42:09] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
0.854(+/-0.015) for {'n_estimators': 20}
0.913(+/-0.016) for {'n_estimators': 50}
0.913(+/-0.018) for {'n_estimators': 100}
0.911(+/-0.018) for {'n_estimators': 150}
0.911(+/-0.019) for {'n_estimators': 200}
0.911(+/-0.019) for {'n_estimators': 300}
0.910(+/-0.019) for {'n_estimators': 400}
0.910(+/-0.019) for {'n_estimators': 500}


In [55]:
#调参 XGboost——maxdepth minchildweight 其他默认
cv_params = [{'max_depth': [3, 5,7,9], 'min_child_weight': [1, 3, 5]}]

scores_GBM_mami=['r2']

for score in scores_GBM_mami:
    print(score)
    model = optimized_GBM.best_estimator_
    optimized_GBM_mami=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM_mami.fit(X_train,y_train)
    
    print(optimized_GBM_mami.best_estimator_)
    
    means_GBM_mami = optimized_GBM_mami.cv_results_['mean_test_score']
    stds_GBM_mami=optimized_GBM_mami.cv_results_['std_test_score']
    params_GBM_mami = optimized_GBM_mami.cv_results_['params']
    
    for (param_GBM_mami,mean_score_GBM_mami,std_score_GBM_mami) in zip(params_GBM_mami,means_GBM_mami,stds_GBM_mami):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM_mami,std_score_GBM_mami,param_GBM_mami))

r2
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    6.0s finished


[16:42:18] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=None, n_estimators=50,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
0.913(+/-0.016) for {'max_depth': 3, 'min_child_weight': 1}
0.914(+/-0.009) for {'max_depth': 3, 'min_child_weight': 3}
0.912(+/-0.016) for {'max_depth': 3, 'min_child_weight': 5}
0.912(+/-0.018) for {'max_depth': 5, 'min_child_weight': 1}
0.914(+/-0.008) for {'max_depth': 5, 'min_child_weight': 3}
0.911(+/-0.018) for {'max_depth': 5, 'min_child_weight': 5}
0.910(+

In [23]:
X_train, X_test, y_train, y_test = train_test_split(dataset_blend_train, y, test_size=0.2, random_state=156)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

model_xgbr_t = optimized_GBM.best_estimator_
model_xgbr_t.fit(X_train, y_train)



print(model_xgbr_t.score(X_test,y_test))

y_pred = model_xgbr_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
model_xgbr = optimized_GBM.best_estimator_

rms = sqrt(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
print(rms)

(25176, 4) (25176,) (6294, 4) (6294,)
[14:32:51] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9217693680580265
0.9217693680580265
0.9217693680580265
21282.752056579666


In [ ]:
#肯定不准确的预测score
X_train, X_test, y_train, y_test = train_test_split(dataset_blend_train, y, test_size=0.2, random_state=156)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

model_stacking_svm.fit(X_train, y_train)

print(model_stacking_svm.score(X_test,y_test))

In [ ]:
clf_st_best = clf_st.best_estimator_

clf_st_best.fit(dataset_blend_train, y)
y_submission = clf_st_best.predict(dataset_blend_test)

#生成文件
submission_df=pd.DataFrame(data={'Id':id_test,'SalePrice':y_submission})
submission_df.head()
submission_df.to_csv('baseline_st3_2.csv',header=False,index=False)